In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


# Define the decision network architecture
class DecisionNetwork(nn.Module):
    def __init__(self, encode_feature_dim, ts_timestep, ts_dim, manual_feature_dim, n_classes=3):
        super(DecisionNetwork, self).__init__()

        self.transformer_encoder_layer = TransformerEncoderLayer(
            d_model=ts_dim, nhead=4)
        self.transformer_encoder = TransformerEncoder(
            self.transformer_encoder_layer, num_layers=2)

        self.fc1 = nn.Linear(encode_feature_dim +
                             ts_dim + manual_feature_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, n_classes)
        # Additional output for transaction percentage
        self.fc4 = nn.Linear(128, 1)

    def forward(self, encode_features, ts_data, manual_features):
        # Apply the transformer encoder to time series data
        transformer_output = self.transformer_encoder(ts_data)

        # Get the last time step output
        last_ts_output = transformer_output[:, -1, :]

        # Concatenate the three inputs
        combined_features = torch.cat(
            (encode_features, last_ts_output, manual_features), dim=1)

        # Pass through the fully connected layers
        x = F.relu(self.fc1(combined_features))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        # Output percentage using sigmoid activation function
        percentage = torch.sigmoid(self.fc4(x))

        return actions, percentage


In [69]:
# Sample data
batch_size = 32
encode_feature_dim = 2048 * 3
ts_timestep = 64
ts_dim = 8
manual_feature_dim = 16

encode_features = torch.randn(batch_size, encode_feature_dim)
ts_data = torch.randn(batch_size, ts_timestep, ts_dim)
manual_features = torch.randn(batch_size, manual_feature_dim)

In [74]:
# Create the decision network and process sample data
decision_network = DecisionNetwork(
    encode_feature_dim, ts_timestep, ts_dim, manual_feature_dim)

with torch.no_grad():
    output = decision_network(encode_features, ts_data, manual_features)


In [75]:
# Process sample data
output_actions, output_percentage = decision_network(encode_features, ts_data, manual_features)

# Get the most likely action
predicted_actions = torch.argmax(output_actions, dim=1)

# Calculate the amount to buy, sell, or hold
current_holdings = 100
transaction_amount = (output_percentage * current_holdings).squeeze()

print("Predicted actions:", predicted_actions)
print("Transaction percentage:", output_percentage)
print("Transaction amount:", transaction_amount)


Predicted actions: tensor([2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
Transaction percentage: tensor([[0.4851],
        [0.4943],
        [0.4714],
        [0.4766],
        [0.4780],
        [0.4956],
        [0.4784],
        [0.5009],
        [0.4764],
        [0.4911],
        [0.5031],
        [0.4828],
        [0.4957],
        [0.5019],
        [0.4734],
        [0.4959],
        [0.5020],
        [0.5029],
        [0.4645],
        [0.4734],
        [0.4888],
        [0.5000],
        [0.5093],
        [0.4858],
        [0.4957],
        [0.4809],
        [0.4946],
        [0.4992],
        [0.4592],
        [0.4899],
        [0.4794],
        [0.5001]], grad_fn=<SigmoidBackward0>)
Transaction amount: tensor([48.5064, 49.4330, 47.1433, 47.6578, 47.7963, 49.5598, 47.8422, 50.0944,
        47.6390, 49.1070, 50.3082, 48.2848, 49.5744, 50.1866, 47.3430, 49.5930,
        50.1999, 50.2855, 46.4516, 47.3414, 48.8757, 50.003